#Imports

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as plt_img
from skimage.color import rgb2gray
from skimage import img_as_ubyte, img_as_float32
import keras
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import math
from PIL import Image,ImageDraw
import argparse
import json
from tensorflow.keras import callbacks
from sklearn import model_selection
from tqdm import trange

In [0]:
#fix random seed
tf.random.set_seed(2342323)
np.random.seed(347786349)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#GPU Info

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [0]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

#Delete Dataset folders

In [0]:
#clean script output
#!rm -rf resized
#!rm -rf heatmaps

#Dataset preparation

In [0]:
#import resized images and heatmaps
!cp /content/drive/My\ Drive/heatmaps.zip /content
!cp /content/drive/My\ Drive/resized.zip /content
!unzip resized.zip -d /content/
!unzip heatmaps.zip -d /content/

In [0]:
#define the function that generates input tensors, to be called in training loop.
dataset_path = "/content"
def preprocessInput(X,Y, start_from, end_to, dataset_path):
    
    for i in trange(start_from, end_to+1):
        #read a hand image from folder and turn it to 8-bit unsigned byte
        resized = plt_img.imread(dataset_path+"/resized/"+str(i)+".jpg")
        resized = resized.reshape(320,320,3)
        resized = img_as_ubyte(resized)
        #append a hand image to X
        X.append(resized)

        #read the first fingertip heatmap
        heatmap1=plt_img.imread(dataset_path+"/heatmaps/"+str(i)+"_1"+".png")
        #turn it in a grayscale representation
        heatmap1 = rgb2gray(heatmap1)
        heatmap1 = heatmap1.reshape(320,320,1)
        heatmap1 = img_as_ubyte(heatmap1)
        for j in range(2,6):
            #do the same for the remaining 4 fingertip heatmaps
            temp = plt_img.imread(dataset_path+"/heatmaps/"+str(i)+'_'+str(j)+".png") #this line should be updated for the new dataset
            temp = rgb2gray(temp)
            temp = temp.reshape(320,320,1)
            temp = img_as_ubyte(temp)
            #concatenate it to the other heatmaps related to single hand image
            heatmap1=tf.concat([heatmap1,temp],-1)
        
        #append the heatmaps as a list of lenght 5 to Y
        Y.append(heatmap1)
    return X , Y

#Model Definition

In [0]:
def cpm(feature_extractor, output_layer_name, trainable_layers):
    
    # get the original input layer tensor
    input_t = feature_extractor.get_layer(index=0).input

    # set the feture extractor layers as non-trainable
    for idx,layer in enumerate(feature_extractor.layers):
      if layer.name in trainable_layers:
        layer.trainable = True
      else:
        layer.trainable = False

    # get the output tensor from a layer of the feature extractor
    vgg_features = feature_extractor.get_layer(name = output_layer_name).output


    #Init output list
    outputs = []

    #------------------------------STAGE-0--------------------------------------
    conv = tf.keras.layers.Conv2D(128, (3,3), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage0_conv2d_1")(vgg_features)
    conv = tf.keras.layers.BatchNormalization()(conv)
    
    #apply resampling to vgg features
    resampled_features_stage0 = tf.keras.layers.UpSampling2D(size=(8,8), interpolation='bilinear')(conv)

    conv = tf.keras.layers.Conv2D(512, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage0_conv2d_2")(resampled_features_stage0)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage0_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization(name = 'out_0')(conv)

    #Append stage output to output list
    #outputs.append(conv) ####output too noisy
    
    #Concatenation Step
    list0= []
    list0.append(resampled_features_stage0)
    list0.append(conv)
    conv = tf.keras.layers.Concatenate(axis=-1)(list0)

    #------------------------------STAGE-1--------------------------------------
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_1")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_2")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_4")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_5")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_6")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage1_conv2d_7")(conv)
    conv = tf.keras.layers.BatchNormalization(name='out_1')(conv)
    
    #Concatenation Step
    list0= []
    list0.append(resampled_features_stage0)
    list0.append(conv)
    #Append stage output to output list
    outputs.append(conv)

    conv = tf.keras.layers.Concatenate(axis=-1)(list0)

    #------------------------------STAGE-2--------------------------------------
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_1")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_2")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_4")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_5")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_6")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage2_conv2d_7")(conv)
    conv = tf.keras.layers.BatchNormalization(name='out_2')(conv)
    
    #Concatenation Step
    list0= []
    list0.append(resampled_features_stage0)
    list0.append(conv)
    #Append stage output to output list
    outputs.append(conv)

    conv = tf.keras.layers.Concatenate(axis=-1)(list0)

    #------------------------------STAGE-3--------------------------------------
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_1")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_2")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_4")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_5")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_6")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage3_conv2d_7")(conv)
    conv = tf.keras.layers.BatchNormalization(name='out_3')(conv)

    

    list0= []
    list0.append(resampled_features_stage0)
    list0.append(conv)
    #Append stage output to output list
    outputs.append(conv)

    conv = tf.keras.layers.Concatenate(axis=-1)(list0)

    #stage4
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_1")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_2")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_4")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_5")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_6")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage4_conv2d_7")(conv)
    conv = tf.keras.layers.BatchNormalization(name='out_4')(conv)

    """list0= []
    list0.append(resampled_features_stage0)
    list0.append(conv)"""
    outputs.append(conv)

    #conv = tf.keras.layers.Concatenate(axis=-1)(list0)

    #stage5
    """conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_1")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_2")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_3")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_4")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (7,7), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_5")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_6")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(5, (1,1), strides=(1, 1), padding='same', activation="relu", use_bias=True, name ="stage5_conv2d_7")(conv)
    conv = tf.keras.layers.BatchNormalization(name='out_5')(conv)"""

    #Append stage output to output list
    #outputs.append(conv)
    
    #set cpm-network layers trainable explicitely 
    # in order to restore optimzer state on model load correctly
    model = tf.keras.Model(input_t, outputs, name="cpm")
    for idx,layer in enumerate(model.layers):
      if idx >=16:
         layer.trainable=True
      
    model.compile(optimizer='adam', loss = tf.keras.losses.mean_squared_error)

    print(model.input_shape)

    return model

#Model instantiation

In [0]:
#Get VGG pretrained on Imagenet
model = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(320,320,3))

In [0]:
#select a layer as features output from VGG
name_output_extractor = "block4_conv4"
#choose wich vgg layers are trainable
trainable_layers = []

#instantiate the CPM
model = cpm(feature_extractor=model, output_layer_name = name_output_extractor, trainable_layers=trainable_layers)

# Model Summary

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [0]:
model.summary()

#Resume Training

In [0]:
!cp /content/drive/My\ Drive/resume.h5 /content/resume.h5
model = tf.keras.models.load_model(filepath='/content/resume.h5')

#Callbacks

In [0]:
#CAMBIARE IN BASE ALLA LUNGHEZZA DELLA RETE (IL NUMERO DELL' OUTPUT)
num_output = 4

filename = '/content/drive/My Drive/cpm.{epoch:02d}-{val_out_%s_loss:.2f}.h5' % (num_output)
print(filename)
monitor = 'val_out_%s_loss' % (num_output)
print(monitor)

#setup the checkpoint callback
checkpoint = callbacks.ModelCheckpoint(os.path.join(filename), monitor=monitor, verbose=1, save_best_only=False, save_weights_only=False, mode='auto')


#Training

In [0]:
##Init input tensor lists
X_train = []
Y_train = []

num_samples = 3188

X_train,Y_train = preprocessInput(X_train,Y_train, 1, num_samples, dataset_path)

##Split Dataset in train and validation sets
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_train, Y_train, test_size=0.33)

print('Dataset split')

training_samples = len(X_train)
test_samples = len(X_test)

print('Training samples: ' + str(training_samples))
print('Test samples: ' + str(test_samples))

# adds a dimension (it becomes a 2D tensor with only one "row")
X_train = tf.stack(X_train)
Y_train = tf.stack(Y_train)

X_test = tf.stack(X_test)
Y_test = tf.stack(Y_test)

# I only want to use this instance of ImageDataGenerator for rescale
datagenTrain = ImageDataGenerator(rescale=1. / 255) 
# Make it so that the data is passed via batches
datagenTrain.fit(X_train)

datagenTest = ImageDataGenerator(rescale=1. / 255) 
datagenTest.fit(X_test)


In [0]:
epochs = 12
initial_epoch = 6
batch_size = 2
steps_per_epoch = training_samples//batch_size
validation_steps = test_samples//batch_size

In [0]:
try:
    # We pass the datagenTrain.flow instead of the full tensor, so that it is rescaled and it is passed by batches
    history = model.fit(datagenTrain.flow(X_train, Y_train, batch_size= batch_size),\
                        steps_per_epoch= steps_per_epoch,\
                        validation_steps = validation_steps,\
                        epochs = epochs,\
                        validation_data = datagenTest.flow(X_test, Y_test, batch_size = batch_size),\
                        callbacks = [checkpoint], \
                        initial_epoch = initial_epoch)
except KeyboardInterrupt:
    pass


#Load Best

In [0]:
#DA FARE CICLO PER LA VALIDATION FINALE
#!cp /content/drive/My\ Drive/cpmBest.h5 /content/cpmBest.h5
#model = tf.keras.models.load_model('/content/drive/My Drive/cpmBest.h5')

#Evaluation

In [0]:
#Pick a sample
##Init input sample tensor lists
X_sample = []
Y_sample = []

#choose a sample for a quick rough evaluation
sample = 167

#get input tensors list
X_sample,Y_sample = preprocessInput(X_sample, Y_sample, sample, sample, dataset_path)
X_sample = tf.stack(X_sample)
Y_sample = tf.stack(Y_sample)

In [0]:
loss_history = history.history['out_%s_loss' % (num_output)]

#Experimental (scale loss values from 0 to 1)
#max_loss = max(loss_history)
#loss_history[:] = [x // max_loss for x in loss_history]

val_loss_history = history.history['val_out_%s_loss' % (num_output)]

total_loss = history.history['loss']

total_val_loss = history.history['val_loss']

#Experimental (scale val_loss values from 0 to 1)
#max_val_loss = max(val_loss_history)
#val_loss_history[:] = [x // max_val_loss for x in val_loss_history]

In [0]:
# Plot training loss values (log scale)
plt.yscale('log')
plt.plot(loss_history)
plt.title('Training loss (log scale)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.savefig('/content/drive/My Drive/out_loss_log.png', bbox_inches='tight')
plt.show()

In [0]:
# Plot training loss values
plt.plot(loss_history)
plt.title('Training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.savefig('/content/drive/My Drive/out_loss.png', bbox_inches='tight')
plt.show()

In [0]:
# Plot validation loss values (log scale)
plt.yscale('log')
plt.plot(val_loss_history, color = 'red')
plt.title('Test loss (log scale)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test'], loc='upper left')
plt.savefig('/content/drive/My Drive/val_out_loss_log.png', bbox_inches='tight')
plt.show()

In [0]:
# Plot validation loss values
plt.plot(val_loss_history, color = 'red')
plt.title('Test loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test'], loc='upper left')
plt.savefig('/content/drive/My Drive/val_out_loss.png', bbox_inches='tight')
plt.show()

In [0]:
# Plot sum of training loss values
plt.plot(total_loss)
plt.title('Training total loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.savefig('/content/drive/My Drive/total_loss.png', bbox_inches='tight')
plt.show()


In [0]:
# Plot sum of validation loss values
plt.plot(total_val_loss, color = 'red')
plt.title('Test total loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test'], loc='upper left')
plt.savefig('/content/drive/My Drive/total_val_loss.png', bbox_inches='tight')
plt.show()

In [0]:
#var = tf.unstack(X_train)
#print(len(var))
#print(var[0][0])
var = X_sample
#var = var.reshape(1,320,320,3)
var=model.predict(var)
#var = var.reshape(320,320,3)
#print(var.shape)
#var = tf.unstack(var)
#print(var)

#var = tf.slice(var[5], [0,0,0,0], [1,320,320,1])
#print(v.shape)

#var = rgb2gray(var)
#var = var.reshape(1,320, 320, 1)

#print(var)
#plt.imshow(tf.keras.preprocessing.image.array_to_img(var[0]),cmap='gray')
w=4
h=5
fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5

#fig.add_subplot(rows, columns, i)
#plt.imshow(tf.keras.preprocessing.image.array_to_img(var[0]),cmap='gray')

for i in range(0, 5):
    var1 = tf.slice(var[3], [0,0,0,i], [1,320,320,1])
    var2=tf.slice(Y_sample, [0,0,0,i], [1,320,320,1])
    #fig.add_subplot(rows, columns, i)
    #plt.imshow(tf.keras.preprocessing.image.array_to_img(var1[0]),cmap='gray')
    size=320
    x_mean_pred=0
    y_mean_pred=0
    tot_sum_pred=0
    img=tf.keras.preprocessing.image.array_to_img(var1[0])
    mat=img.load() 
    img1=tf.keras.preprocessing.image.array_to_img(var2[0])
    #im = Image.new('RGB', (320, 320), (128, 128, 128))
    draw = ImageDraw.Draw(img)
    for r in range(0,size):
        for c in range(0,size):
            if mat[r,c]>200:
               tot_sum_pred=tot_sum_pred+mat[r,c]
               x_mean_pred=x_mean_pred+c*mat[r,c]
               y_mean_pred=y_mean_pred+r*mat[r,c]
    if tot_sum_pred!=0:        
       x_mean_pred=x_mean_pred/tot_sum_pred
       y_mean_pred=y_mean_pred/tot_sum_pred
 
       draw.ellipse((y_mean_pred, x_mean_pred, y_mean_pred+35, x_mean_pred+35), fill = 'green', outline ='green')
 
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img,cmap='gray')
plt.show()

In [0]:
#var = tf.unstack(X_train)
#print(len(var))
#print(var[0][0])
var = X_sample
#var = var.reshape(1,320,320,3)
var=model.predict(var)
#var = var.reshape(320,320,3)
#print(var.shape)
#var = tf.unstack(var)
#print(var)

#var = tf.slice(var[5], [0,0,0,0], [1,320,320,1])
#print(v.shape)

#var = rgb2gray(var)
#var = var.reshape(1,320, 320, 1)

#print(var)
#plt.imshow(tf.keras.preprocessing.image.array_to_img(var[0]),cmap='gray')
w=4
h=5
fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5

#fig.add_subplot(rows, columns, i)
#plt.imshow(tf.keras.preprocessing.image.array_to_img(var[0]),cmap='gray')

for i in range(0, 5):
    var1 = tf.slice(var[3], [0,0,0,i], [1,320,320,1])
    var2=tf.slice(Y_sample, [0,0,0,i], [1,320,320,1])
    #fig.add_subplot(rows, columns, i)
    #plt.imshow(tf.keras.preprocessing.image.array_to_img(var1[0]),cmap='gray')
    size=320
    x_mean_pred=0
    y_mean_pred=0
    tot_sum_pred=0
    img=tf.keras.preprocessing.image.array_to_img(var1[0])
    mat=img.load() 
    img1=tf.keras.preprocessing.image.array_to_img(var2[0])
    #im = Image.new('RGB', (320, 320), (128, 128, 128))
    draw = ImageDraw.Draw(img1)
    for r in range(0,size):
        for c in range(0,size):
            if mat[r,c]>200:
               tot_sum_pred=tot_sum_pred+mat[r,c]
               x_mean_pred=x_mean_pred+c*mat[r,c]
               y_mean_pred=y_mean_pred+r*mat[r,c]
    if tot_sum_pred!=0:        
       x_mean_pred=x_mean_pred/tot_sum_pred
       y_mean_pred=y_mean_pred/tot_sum_pred
 
       draw.ellipse((y_mean_pred, x_mean_pred, y_mean_pred+35, x_mean_pred+35), fill = 'green', outline ='green')
 
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img1,cmap='gray')
plt.show()

In [0]:
import statistics

#Accuracy function
def accuracy(Y_pred, Y_test, num_samples, treshold):
    accuracy = []

    for n in trange(0, num_samples):
        coords_pred = []
        coords_true = []

        for h in range(0, 5):

            sum_pred = 0 
            sum_true = 0
            x_pred = 0
            x_true = 0
            y_pred = 0
            y_true = 0
            pred = tf.slice(Y_pred, [n,0,0,h], [1,320,320,1])
            true =tf.slice(Y_test, [n,0,0,h], [1,320,320,1])
            pred = tf.keras.preprocessing.image.array_to_img(pred[0])
            true = tf.keras.preprocessing.image.array_to_img(true[0])
            pred = pred.load()
            true = true.load()

            for i in range(0,320):
                for j in range(0,320):
                    sum_pred = sum_pred + pred[i,j]
                    x_pred = x_pred + j*pred[i,j]
                    y_pred = y_pred + i*pred[i,j]

                    sum_true = sum_true + true[i,j]
                    x_true = x_true + j*true[i,j]
                    y_true = y_true + i*true[i,j]
            
            if(sum_pred != 0 and sum_true !=0):
                x_pred = x_pred/sum_pred
                y_pred = y_pred/sum_pred
                x_true = x_true/sum_true
                y_true = y_true/sum_true

            coords_pred.append((x_pred, y_pred))
            coords_true.append((x_true, y_true))

        #compute accuracy on coordinates
        for to_check in range(0,5):
            x_pred = coords_pred[to_check][0]
            x_true = coords_true[0][0]
            y_pred = coords_pred[to_check][1]
            y_true = coords_true[0][1]
                
            min_dist_index = 0


            min_dist = math.sqrt(math.pow((x_true-x_pred),2) + math.pow((y_true-y_pred), 2))

            for neighbor in range(1,5):
                x_pred = coords_pred[to_check][0]
                x_true = coords_true[neighbor][0]
                y_pred = coords_pred[to_check][1]
                y_true = coords_true[neighbor][1]
                    
                dist = math.sqrt(math.pow((x_true-x_pred),2) + math.pow((y_true-y_pred), 2))

                if dist < min_dist:
                    min_dist = dist
                    min_dist_index = neighbor                        
                
            if to_check == min_dist_index and min_dist < treshold:
                accuracy.append(1)
            else:
                accuracy.append(0)
                    
        sum_pred = 0
        sum_true = 0
        x_pred = 0
        x_true = 0
        y_pred = 0
        y_true = 0
    
    return statistics.mean(accuracy)
                    
                

                    

In [0]:
Y_pred = model.predict(X_test)

In [0]:
Y = Y_pred[4]
print(len(Y_pred))
print(Y.shape)
print(Y.shape)

In [0]:
print(accuracy(Y,Y_test, 78, 14))

In [0]:
#Final Evaluation
!cp /content/drive/My\ Drive/modelli_accuracy/cpm01.h5 /content/cpm01.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm02.h5 /content/cpm02.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm03.h5 /content/cpm03.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm04.h5 /content/cpm04.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm05.h5 /content/cpm05.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm06.h5 /content/cpm06.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm07.h5 /content/cpm07.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm08.h5 /content/cpm08.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm09.h5 /content/cpm09.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm10.h5 /content/cpm10.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm11.h5 /content/cpm11.h5
!cp /content/drive/My\ Drive/modelli_accuracy/cpm12.h5 /content/cpm12.h5



In [0]:

import statistics

#Strict Accuracy function
def strict_accuracy(Y_pred, Y_test, num_samples, treshold):
    accuracy = []

    for n in range(0, num_samples):
        #init array of coordinates
        coords_pred = []
        coords_true = []

        for h in range(0, 5):
            
            #init
            sum_pred = 0 
            sum_true = 0
            x_pred = 0
            x_true = 0
            y_pred = 0
            y_true = 0

            #get sample from test set
            pred = tf.slice(Y_pred, [n,0,0,h], [1,320,320,1])
            true =tf.slice(Y_test, [n,0,0,h], [1,320,320,1])
            pred = tf.keras.preprocessing.image.array_to_img(pred[0])
            true = tf.keras.preprocessing.image.array_to_img(true[0])
            pred = pred.load()
            true = true.load()

            #extract the predicted and ground-truth coordinates
            for i in range(0,320):
                for j in range(0,320):
                    sum_pred = sum_pred + pred[i,j]
                    x_pred = x_pred + j*pred[i,j]
                    y_pred = y_pred + i*pred[i,j]

                    sum_true = sum_true + true[i,j]
                    x_true = x_true + j*true[i,j]
                    y_true = y_true + i*true[i,j]
            
            if(sum_pred != 0 and sum_true !=0):
                x_pred = x_pred/sum_pred
                y_pred = y_pred/sum_pred
                x_true = x_true/sum_true
                y_true = y_true/sum_true

            coords_pred.append((x_pred, y_pred))
            coords_true.append((x_true, y_true))

        #compute accuracy on coordinates
        for to_check in range(0,5):
            #check if the predicted coordinate is nearer to the ground truth coordinate
            #by checking the distance from all other hand keypoints 
            x_pred = coords_pred[to_check][0]
            x_true = coords_true[0][0]
            y_pred = coords_pred[to_check][1]
            y_true = coords_true[0][1]
                
            min_dist_index = 0


            min_dist = math.sqrt(math.pow((x_true-x_pred),2) + math.pow((y_true-y_pred), 2))

            for neighbor in range(1,5):
                x_pred = coords_pred[to_check][0]
                x_true = coords_true[neighbor][0]
                y_pred = coords_pred[to_check][1]
                y_true = coords_true[neighbor][1]
                    
                dist = math.sqrt(math.pow((x_true-x_pred),2) + math.pow((y_true-y_pred), 2))

                if dist < min_dist:
                    min_dist = dist
                    min_dist_index = neighbor                        
            
            #actually check if the computed minimum distance is related to the ground truth
            #also check wheter or not is below a certain pixel treshold 
            if to_check == min_dist_index and min_dist < treshold:
                accuracy.append(1)
            else:
                accuracy.append(0)
                    
    return statistics.mean(accuracy)
    

In [0]:
#Accuracy function
def accuracy(Y_pred, Y_test, num_samples, treshold):
    accuracy = []

    for n in range(0, num_samples):
        #init array of coordinates
        coords_pred = []
        coords_true = []

        for h in range(0, 5):
            
            #init
            sum_pred = 0 
            sum_true = 0
            x_pred = 0
            x_true = 0
            y_pred = 0
            y_true = 0

            #get sample from test set
            pred = tf.slice(Y_pred, [n,0,0,h], [1,320,320,1])
            true =tf.slice(Y_test, [n,0,0,h], [1,320,320,1])
            pred = tf.keras.preprocessing.image.array_to_img(pred[0])
            true = tf.keras.preprocessing.image.array_to_img(true[0])
            pred = pred.load()
            true = true.load()

            #extract the predicted and ground-truth coordinates
            for i in range(0,320):
                for j in range(0,320):
                    sum_pred = sum_pred + pred[i,j]
                    x_pred = x_pred + j*pred[i,j]
                    y_pred = y_pred + i*pred[i,j]

                    sum_true = sum_true + true[i,j]
                    x_true = x_true + j*true[i,j]
                    y_true = y_true + i*true[i,j]
            
            if(sum_pred != 0 and sum_true !=0):
                x_pred = x_pred/sum_pred
                y_pred = y_pred/sum_pred
                x_true = x_true/sum_true
                y_true = y_true/sum_true

            coords_pred.append((x_pred, y_pred))
            coords_true.append((x_true, y_true))

        #compute accuracy on coordinates
        for to_check in range(0,5):
            x_pred = coords_pred[to_check][0]
            x_true = coords_true[to_check][0]
            y_pred = coords_pred[to_check][1]
            y_true = coords_true[to_check][1]

            dist = math.sqrt(math.pow((x_true-x_pred),2) + math.pow((y_true-y_pred), 2))                     
            
            #actually check whether the computed 2D distance is below a certain number of pixel treshold 
            if dist < treshold:
                accuracy.append(1)
            else:
                accuracy.append(0)
                    
    return statistics.mean(accuracy)

In [0]:
treshold = 50
test_samples = 1053

mini_test = 81

num_coordinates = test_samples*5

accuracy_values = []

for i in trange(1,13):
    temp_accuracy = 0
    model = tf.keras.models.load_model('/content/cpm%02d.h5'%(i))
    print("loaded model %d" %(i))
    for j in range(0, (test_samples//mini_test)):
        X_mini_test = tf.slice(X_test, [j*(mini_test-1),0,0,0],[81,320,320,3])
        Y_pred = model.predict(X_mini_test)

        Y = Y_pred[num_output - 1]

        Y_mini_test = tf.slice(Y_test, [j*(mini_test-1),0,0,0],[81,320,320,5])
        acc = accuracy(Y, Y_mini_test, mini_test, treshold)
        print(acc)
        temp_accuracy += acc*mini_test*5
    
    temp_accuracy = temp_accuracy/num_coordinates
    print("accuracy epoch"+ str(i)+ ": "+ str(temp_accuracy))
    accuracy_values.append(temp_accuracy)

In [0]:
#plot accuracy at each epoch
plt.bar(range(1,13), accuracy_values)
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.savefig('/content/drive/My Drive/accuracy.png', bbox_inches='tight')
plt.show()